Messing around with calculation of Greatest Common Factor between `n` numbers

Sparked by [this answer](https://www.quora.com/My-US-citizen-boyfriend-who-is-a-software-engineer-with-a-MS-in-Computer-Science-is-unable-to-get-a-job-in-his-field-Is-it-because-he-is-being-undermined-by-people-on-H1Bs/answers/38917295?srid=oDGD) to [this question](https://www.quora.com/My-US-citizen-boyfriend-who-is-a-software-engineer-with-a-MS-in-Computer-Science-is-unable-to-get-a-job-in-his-field-Is-it-because-he-is-being-undermined-by-people-on-H1Bs) on Quora

In [1]:
search_vals = [1600, 1200, 800]

def get_gcf(func, params):
    
    sets = map(func, map(abs,params))
    temp = reduce(lambda x, y: list(set(x).intersection(y)), sets)
    
    return max(temp)

# End get_gcf()

Naive Python attempt

In [2]:
def naive_py(val):
    factors = [1, val]
    for x in range(2, val):
        for y in range(x, val):
            if (x * y) == val:
                factors.extend([x,y])
            # End if
        # End for
    # End for
    
    return factors
# End naive_py()

In [3]:
%timeit get_gcf(naive_py, search_vals)
get_gcf(naive_py, search_vals)

10 loops, best of 3: 102 ms per loop


400

Using generators and list comprehensions...

In [4]:
def improved_py(val):
    factors = [1, val]
    [factors.extend([x, y]) 
         for x in xrange(2, val) 
         for y in xrange(x, val) if (x * y) == val]
    
    return factors
# End improved_py()

In [5]:
%timeit get_gcf(improved_py, search_vals)
get_gcf(improved_py, search_vals)

10 loops, best of 3: 93.1 ms per loop


400

Using Cython

In [6]:
%load_ext Cython

In [7]:
%%cython
def cython_factors(int val):
    cdef int x, y
    cdef list factors = [1, val]
    
    for x in range(2, val):
        for y in range(x, val):
            if (x * y) == val:
                factors.extend([x, y])
            # End if
        # End for
    # End for
    
    return factors
# End cython_factors()

In [8]:
%timeit get_gcf(cython_factors, search_vals)
get_gcf(cython_factors, search_vals)

1000 loops, best of 3: 1.53 ms per loop


400

Using numpy arrays does not help in this case (in fact, it slows it down) because `np.append` recreates the array every loop

In [9]:
%%cython
import numpy as np
cimport numpy as np
def cython_factors2(int val):
    cdef int x, y
    cdef np.ndarray[int] factors = np.array([1, val])
    
    for x in range(2, val):
        for y in range(x, val):
            if (x * y) == val:
                factors = np.append(factors, [x, y])
            # End if
        # End for
    # End for
    
    return factors
# End cython_factors2()

In [10]:
%timeit get_gcf(cython_factors2, search_vals)
get_gcf(cython_factors2, search_vals)

1000 loops, best of 3: 1.69 ms per loop


400

List comprehensions get compiled to the same C code as Cython for loops (timings will be the same or very similar)

In [11]:
%%cython
import numpy as np
cimport numpy as np
def cython_factors3(int val):
    cdef int x, y
    cdef list factors = [1, val]

    [factors.extend([x, y]) 
         for x in xrange(2, val) 
         for y in xrange(x, val) if (x * y) == val]
    
    return factors
# End cython_factors3()

In [12]:
%timeit get_gcf(cython_factors3, search_vals)
get_gcf(cython_factors3, search_vals)

1000 loops, best of 3: 1.53 ms per loop


400

How does Cython scale compared to the improved Python version?

In [13]:
expanded = search_vals + [8000, 7260, 9800, 6520]
expanded

[1600, 1200, 800, 8000, 7260, 9800, 6520]

In [14]:
%timeit get_gcf(improved_py, expanded)
get_gcf(improved_py, expanded)

1 loop, best of 3: 5.23 s per loop


20

In [15]:
%timeit get_gcf(cython_factors3, expanded)
get_gcf(cython_factors3, expanded)

10 loops, best of 3: 83.4 ms per loop


20

Greatest efficiency gains come from using an appropriate algorithm...

In [16]:
def modulo_py(val):
    factors = [1, val]
    for x in xrange(2, val):
        if (val % x) == 0:
            factors.append(x)
        # End if
    # End for
    
    return factors
# End modulo_py()

In [17]:
%%cython 
def modulo_cy(int val):
    cdef list factors = [1, val]
    cdef int x
    
    for x in xrange(2, val):
        if (val % x) == 0:
            factors.append(x)
        # End if
    # End for
    
    return factors
# End modulo_cy()

In [18]:
%timeit get_gcf(modulo_py, expanded)
get_gcf(modulo_py, expanded)

1000 loops, best of 3: 1.52 ms per loop


20

In [19]:
%timeit get_gcf(modulo_cy, expanded)
get_gcf(modulo_cy, expanded)

10000 loops, best of 3: 167 µs per loop


20

We could speed things up by sorting the list first from smallest to largest value and abandoning subsequent searches if a value larger than anything in the list of factors for the smallest value is found

In [20]:
def improved_modulo_py(val, limit=0):
    factors = [1, val]
    for x in xrange(2, val):
        if limit > 0 and x > limit:
            return factors
            
        if (val % x) == 0:
            factors.append(x)
        # End if
    # End for
    
    return factors
# End improved_modulo_py()

In [21]:
%timeit get_gcf(improved_modulo_py, expanded)
get_gcf(improved_modulo_py, expanded)

100 loops, best of 3: 1.99 ms per loop


20

Cost of repeatedly checking the `if` statement is ~50-60ms compared to not checking for a limit

In [22]:
%%cython
def improved_modulo_cy(int val, int limit=0):
    cdef list factors = [1, val]
    cdef int x
    
    for x in xrange(2, val):
        if limit > 0 and x > limit:
            return factors
            
        if (val % x) == 0:
            factors.append(x)
        # End if
    # End for
    
    return factors
# End improved_modulo_cy()

In [23]:
def mod_get_gcf(func, params):
    params = map(abs, params)
    params.sort()
    sets = [func(params[0])]
    limit = max(sets[0])

    sets.extend([func(params[1], limit))
    temp = reduce()
    temp = reduce(lambda x, y: list(set(x).intersection(y)), sets)

    return max(temp)
# End mod_get_gcf()

In [24]:
%timeit mod_get_gcf(improved_modulo_py, expanded)
mod_get_gcf(improved_modulo_py, expanded)

1000 loops, best of 3: 397 µs per loop


20

In [25]:
%timeit mod_get_gcf(improved_modulo_cy, expanded)
mod_get_gcf(improved_modulo_cy, expanded)

10000 loops, best of 3: 48.5 µs per loop


20

Recursion example

Python lacks tail call optimization so recursion is often slow. 

For this particular (simple) case, it is much faster.

In [26]:
def recursive(x1, x2):
    if x2 == 0:
        return x1
    
    return recursive(x2, x1 % x2)
# End recursive()

abs_exp = [abs(i) for i in expanded]
%timeit reduce(recursive, abs_exp)

100000 loops, best of 3: 2.64 µs per loop


In [27]:
reduce(recursive, abs_exp)

20

In [28]:
%%cython
def recursive_cy(int x1, int x2):
    if x2 == 0:
        return x1
    
    return recursive_cy(x2, x1 % x2)
# End recursive_cy()

In [29]:
abs_exp = [abs(i) for i in expanded]
%timeit reduce(recursive_cy, abs_exp)

The slowest run took 5.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.32 µs per loop
